In [4]:
# Importación de librerías
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
import matplotlib.pyplot as plt
from streamlit_option_menu import option_menu
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import pydeck as pdk
# IMPORTACIÓN DE LA BASE DE DATOS
archivo = 'Ejercicio Ingeniero de datos.xlsx'

df_original = pd.read_excel(archivo, sheet_name='Ejercicio 1', header = 4)
  
df_original = df_original.rename(columns={'2015p/': 2015})

# Informaciónn adicional de la consulta de la base de datos
fuente = df_original.iloc[135, 0]
fuente = fuente[7:]
fecha_consulta = df_original.iloc[137, 0]
fecha_consulta = fecha_consulta[19:]  

# Información sobre los estados: latitud y longitud
informacion_estados = pd.read_excel("Latitud y longitud de los estados.xlsx")
informacion_estados = informacion_estados.sort_values(by="Entidad")
informacion_estados = informacion_estados.reset_index(drop=True)

#------------------------------------------------------------------------------------------------#
# TRANSFORMACIÓN DE LOS DATOS
# Redefinición del dataframe
df_original = df_original.iloc[:132,:]

# Crosstable
df_original = pd.melt(df_original, id_vars=['Actividad económica', 'Entidad','Concepto'], 
                    value_vars=[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016],
                    ignore_index=True,
                    var_name='Año', 
                    value_name='PIB')

# Cambiar el formato a la columna año
df_original['Año'] = df_original['Año'].astype('int64')

# Quitar los espacios en blanco al inicio y al final de la columna de entidades
df_original['Entidad'] = df_original['Entidad'].str.strip()

# Sustituir valores de algunos estados
df_original['Entidad'].replace({'Michoacán de Ocampo': 'Michoacán',
                                                        'Veracruz de Ignacio de la Llave':'Veracruz',
                                                        'Coahuila de Zaragoza':'Coahuila'}, inplace=True)


# Unión con la latitud y longitud de los estados
df_original = df_original.merge(informacion_estados, left_on='Entidad', right_on='Entidad')

#df = df_original[['Actividad económica', 'Entidad','Año','PIB']]
df_original

,Actividad económica,Entidad,Concepto,Año,PIB,Latitud,Longitud,Code
0,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2003,121197.634,21.77111,-102.35861,MX-AGU
1,Actividades primarias,Aguascalientes,Aguascalientes -Actividades primarias,2003,5262.297,21.77111,-102.35861,MX-AGU
2,Actividades secundarias,Aguascalientes,Aguascalientes -Actividades secundarias,2003,43925.787,21.77111,-102.35861,MX-AGU
3,Actividades terciarias,Aguascalientes,Aguascalientes -Actividades terciarias,2003,72009.550,21.77111,-102.35861,MX-AGU
4,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2004,126554.147,21.77111,-102.35861,MX-AGU
...,...,...,...,...,...,...,...,...
1787,Actividades terciarias,Zacatecas,Zacatecas -Actividades terciarias,2015,83110.247,22.77093,-102.58325,MX-ZAC
1788,Total de la actividad económica,Zacatecas,Zacatecas -Total de la actividad económica,2016,157897.806,22.77093,-102.58325,MX-ZAC
1789,Actividades primarias,Zacatecas,Zacatecas -Actividades primarias,2016,14114.613,22.77093,-102.58325,MX-ZAC
1790,Actividades secundarias,Zacatecas,Zacatecas -Actividades secundarias,2016,58295.186,22.77093,-102.58325,MX-ZAC


In [7]:
df_original_estado = df_original[df_original["Entidad"]=="Aguascalientes"]
df_original_estado = df_original_estado[df_original_estado["Actividad económica"]=="Total de la actividad económica"]
df_original_estado

,Actividad económica,Entidad,Concepto,Año,PIB,Latitud,Longitud,Code
0,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2003,121197.634,21.77111,-102.35861,MX-AGU
4,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2004,126554.147,21.77111,-102.35861,MX-AGU
8,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2005,129628.028,21.77111,-102.35861,MX-AGU
12,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2006,138111.823,21.77111,-102.35861,MX-AGU
16,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2007,150305.117,21.77111,-102.35861,MX-AGU
20,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2008,150949.983,21.77111,-102.35861,MX-AGU
24,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2009,143253.947,21.77111,-102.35861,MX-AGU
28,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2010,152205.212,21.77111,-102.35861,MX-AGU
32,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2011,158934.494,21.77111,-102.35861,MX-AGU
36,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2012,167705.967,21.77111,-102.35861,MX-AGU


In [18]:
df_original_estado.reset_index(drop=True, inplace=True)
for i in range(df_original_estado["Año"].shape[0] - 1):
    print("PIB año {}: {}".format(df_original_estado["Año"][i],df_original_estado["PIB"][i]))
    print("PIB año {}: {}".format(df_original_estado["Año"][i+1],df_original_estado["PIB"][i+1]))
    a_1 = df_original_estado["PIB"][i]
    a_2 = df_original_estado["PIB"][i+1]
    df_original_estado.loc[i, "delta_PIB"] = (a_2 - a_1)/a_1
    print(df_original_estado.iloc[i, "delta_PIB"])
    print("Nuevo ciclo")

PIB año 2003: 121197.634
PIB año 2004: 126554.147


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [27]:
informacion_estados

,Entidad,Latitud,Longitud,Code
0,Aguascalientes,21.771110,-102.358610,MX-AGU
1,Baja California,32.613270,-115.342280,MX-BCN
2,Baja California Sur,24.072500,-110.362220,MX-BCS
3,Campeche,19.486570,-90.373220,MX-CAM
4,Chiapas,16.711650,-93.111210,MX-CHP
5,Chihuahua,28.150960,-106.212780,MX-CHH
6,Ciudad de México,19.428470,-99.127660,MX-CMXC
7,Coahuila,25.238920,-103.185920,MX-COA
8,Colima,19.147360,-103.722440,MX-COL
9,Durango,24.057180,-104.533200,MX-DUR


In [10]:
df['Año'] = df['Año'].astype('int64')

In [11]:
entidades = df['Entidad'].unique()
entidades

array(['Total nacional ', 'Aguascalientes ', 'Baja California ',
       'Baja California Sur ', 'Campeche ', 'Coahuila de Zaragoza ',
       'Colima ', 'Chiapas ', 'Chihuahua ', 'Ciudad de México ',
       'Durango ', 'Guanajuato ', 'Guerrero ', 'Hidalgo ', 'Jalisco ',
       'México ', 'Michoacán de Ocampo ', 'Morelos ', 'Nayarit ',
       'Nuevo León ', 'Oaxaca ', 'Puebla ', 'Querétaro ', 'Quintana Roo ',
       'San Luis Potosí ', 'Sinaloa ', 'Sonora ', 'Tabasco ',
       'Tamaulipas ', 'Tlaxcala ', 'Veracruz de Ignacio de la Llave ',
       'Yucatán ', 'Zacatecas '], dtype=object)

In [13]:
import numpy as np

chart_data = pd.DataFrame(
   np.random.randn(1000, 2) / [50, 50] + [37.76, -122.4],
   columns=['lat', 'lon'])

st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=20,
        longitude=-120.4,
        zoom=5,
        pitch=50,
    ),
    layers=[
        pdk.Layer(
           'HexagonLayer',
           data=chart_data,
           get_position='[lon, lat]',
           radius=200,
           elevation_scale=4,
           elevation_range=[0, 1000],
           pickable=True,
           extruded=True,
        ),
        pdk.Layer(
            'ScatterplotLayer',
            data=chart_data,
            get_position='[lon, lat]',
            get_color='[200, 30, 0, 160]',
            get_radius=200,
        ),
    ],
))

NameError: name 'st' is not defined

In [17]:
import pydeck as pdk

# 2014 locations of car accidents in the UK
UK_ACCIDENTS_DATA = ('https://raw.githubusercontent.com/uber-common/'
                     'deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv')

# Define a layer to display on a map
layer = pdk.Layer(
    'HexagonLayer',
    df,
    get_position=['Longitud', 'Latitud'],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,                 
    coverage=1)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-120,
    latitude=20,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('demo.html')

In [30]:
df_original['Longitud']

KeyError: 'Longitud'

In [35]:
# Generamos la figura del mapa
fig = go.Figure()
 
# Recorremos los datos del archivo csv y extraemos sus coordenadas,
# etiquetas a mostrar y especificamos características del marcador
for i in df_original:
    fig.add_trace(go.Scattergeo(
            lon = df_original['Longitud'],
            lat = df_original['Latitud'],
            text = df_original['Entidad'],
            marker = dict(
                size = df_original['PIB'],
                color = 'lightseagreen',
                line_color='black',
                line_width=0.5,
                sizemode = 'area'
        )))
 
# Establecemos las características del mapa base, el título y opciones
# de leyenda, así como la propia proyección del mapa.
fig.update_layout(
    title_text = 'Población en las ciudades del mundo, año 2025',
    showlegend = True,
    geo = dict(
        scope="north america",
        resolution=110,
        projection_type='mollweide',
        showcoastlines=True,
        showocean=True,
        oceancolor='#132630',
        lakecolor='#132630',
        coastlinecolor='#224354',
        landcolor='#224354',
    )
)
 
# Mostramos la figura creada con las características dadas
fig.show()

In [38]:
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df_original, 
                        lat="Latitud", 
                        lon="Longitud", 
                        hover_name="Entidad", 
                        hover_data=["Entidad", "PIB"],
                        color="PIB",
                        color_continuous_scale=color_scale,
                        size="PIB",
                        zoom=3, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [22]:
archivo = 'Ejercicio Ingeniero de datos.xlsx'
df_original = pd.read_excel(archivo, sheet_name='Ejercicio 1', header = 4)
 
df_original = df_original.rename(columns={'2015p/': 2015})

# Información sobre los estados: latitud y longitud
informacion_estados = pd.read_csv("Latitud y longitud de los estados.txt", sep=",")
informacion_estados = informacion_estados.sort_values(by="Entidad")
informacion_estados = informacion_estados.reset_index(drop=True)

#------------------------------------------------------------------------------------------------#
# TRANSFORMACIÓN DE LOS DATOS
# Redefinición del dataframe
df_original = df_original.iloc[:132,:]

# Crosstable
df_original = pd.melt(df_original, id_vars=['Actividad económica', 'Entidad','Concepto'], 
                    value_vars=[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016],
                    ignore_index=True,
                    var_name='Año', 
                    value_name='PIB')

# Cambiar el formato a la columna año
df_original['Año'] = df_original['Año'].astype('int64')

# Quitar los espacios en blanco al inicio y al final de la columna de entidades
df_original['Entidad'] = df_original['Entidad'].str.strip()

# Unión con la latitud y longitud de los estados
df_original.merge(informacion_estados, on='Entidad', how='left')

df = df_original[['Actividad económica', 'Entidad','Año','PIB']]

In [24]:
MEX_PIB_DATA = ('https://raw.githubusercontent.com/uber-common/'
                'https://github.com/alexisalamea271828/pib_mexico/blob/7ec811ee43f7130287c640719621eca64fefea62/Latitud%20y%20longitud%20de%20los%20estados.csv')

# Define a layer to display on a map
layer = pdk.Layer(
    'HexagonLayer',
    MEX_PIB_DATA,
    get_position=['Longitud', 'Latitud'],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,                 
    coverage=1)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('demo.html')


In [25]:
st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=20,
        longitude=-100.4,
        zoom=11,
        pitch=50,
    )))

NameError: name 'st' is not defined

In [8]:
informacion_estados = pd.read_csv("Latitud y longitud de los estados.txt", sep=",")
informacion_estados = informacion_estados.sort_values(by="Estado")
informacion_estados = informacion_estados.reset_index(drop=True)
informacion_estados

,Estado,Latitud,Longitud
0,Aguascalientes,21.771110,-102.358610
1,Baja California,32.613270,-115.342280
2,Baja California Sur,24.072500,-110.362220
3,Campeche,19.486570,-90.373220
4,Chiapas,16.711650,-93.111210
5,Chihuahua,28.150960,-106.212780
6,Ciudad de México,19.428470,-99.127660
7,Coahuila de Zaragoza,25.238920,-103.185920
8,Colima,19.147360,-103.722440
9,Durango,24.057180,-104.533200


In [12]:
df_2 = df[['Actividad económica', 'Entidad']]
df_2

,Actividad económica,Entidad
0,Total de la actividad económica,Total nacional
1,Total de la actividad económica,Aguascalientes
2,Total de la actividad económica,Baja California
3,Total de la actividad económica,Baja California Sur
4,Total de la actividad económica,Campeche
...,...,...
133,NaN,NaN
134,Cifras preliminares: p/ A partir de 2015,NaN
135,Fuente: INEGI. Sistema de Cuentas Nacionales d...,NaN
136,NaN,NaN


In [4]:
import plotly.graph_objects as go
import plotly.express as px
import requests
repo_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json' 
#Archivo GeoJSON
mx_regions_geo = requests.get(repo_url).json()

fig = px.choropleth(data_frame=df, 
                    geojson=mx_regions_geo, 
                    locations=df['Entidad'], # nombre de la columna del Dataframe
                    featureidkey='properties.name',  # ruta al campo del archivo GeoJSON con el que se hará la relación (nombre de los estados)
                    color=df['PIB'], #El color depende de las cantidades
                    color_continuous_scale="burg",
                    #scope="north america"
                   )

fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
fig.show()

NameError: name 'df' is not defined

In [2]:
fig = go.Figure(go.Choropleth(
    locations = datos_pob['Country'],
    locationmode = "country names",
    z = datos_pob['Urban_pop_2015'],
    text = datos_pob['Country'],
    colorscale = 'viridis',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='#efefef',
    marker_line_width=0.5,
    colorbar_ticksuffix = '%',
    colorbar_title = 'Población urbana',
    )
)

NameError: name 'go' is not defined

In [56]:
mx_regions_geo.Feature

AttributeError: 'dict' object has no attribute 'Feature'

In [6]:
# Generamos la figura y establecemos los valores para configurar
# el encuadre, la paleta de colores, la capa de datos, la etiqueta
fig = go.Figure(go.Choropleth(
    locations = df['Entidad'],
    locationmode = "ISO-3",
    z = df['PIB'],
    text = df['Entidad'],
    colorscale = 'viridis',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='#efefef',
    marker_line_width=0.5,
    colorbar_ticksuffix = '%',
    colorbar_title = 'Población urbana',
    )
)
 
# Establecemos las características del título y la apariencia del mapa base
fig.update_layout(
    title_text = 'Porcentaje de población urbana en Europa por país, año 2015',
    showlegend = False,
    geo = dict(
        scope='europe',
        resolution=50,
        projection_type='miller',
        showcoastlines=True,
        showocean=True,
        showcountries=True,
        oceancolor='#eaeaea',
        lakecolor='#eaeaea',
        coastlinecolor='#dadada'
    )
)
 
# Mostramos la figura generada
fig.show()

KeyError: 'PIB'

In [3]:
import pandas as pd
import numpy as np

chart_data = pd.DataFrame(
   {"col1": list(range(20)), "col2": np.random.randn(20), "col3": np.random.randn(20)}
)

chart_data


,col1,col2,col3
0,0,1.441093,0.693052
1,1,1.138817,0.117399
2,2,-0.867842,0.883942
3,3,0.158150,-0.742798
4,4,0.232778,-0.202960
5,5,-1.026772,2.177564
6,6,0.155985,1.179628
7,7,1.779176,-0.915092
8,8,1.112467,-0.737008
9,9,0.517811,1.012308


In [61]:
archivo = 'Ejercicio Ingeniero de datos.xlsx'

df_original = pd.read_excel(archivo, sheet_name='Ejercicio 1', header = 4)
df_original = df_original.rename(columns={'2015p/': 2015})

df_original['Entidad'].replace({'Michoacán de Ocampo': 'Michoacán',
                                                        'Veracruz de Ignacio de la Llave':'Veracruz',
                                                        'Coahuila de Zaragoza':'Coahuila'}, inplace=True)

a = df_original['Entidad'].unique()           
a                                           

array(['Total nacional ', 'Aguascalientes ', 'Baja California ',
       'Baja California Sur ', 'Campeche ', 'Coahuila de Zaragoza ',
       'Colima ', 'Chiapas ', 'Chihuahua ', 'Ciudad de México ',
       'Durango ', 'Guanajuato ', 'Guerrero ', 'Hidalgo ', 'Jalisco ',
       'México ', 'Michoacán de Ocampo ', 'Morelos ', 'Nayarit ',
       'Nuevo León ', 'Oaxaca ', 'Puebla ', 'Querétaro ', 'Quintana Roo ',
       'San Luis Potosí ', 'Sinaloa ', 'Sonora ', 'Tabasco ',
       'Tamaulipas ', 'Tlaxcala ', 'Veracruz de Ignacio de la Llave ',
       'Yucatán ', 'Zacatecas ', nan], dtype=object)